Import libraries

In [1]:
import pandas as pd
import numpy as np # library to handle data in a vectorized manner

import geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!pip install folium
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


In [2]:
boroughs=pd.read_csv('/home/sedna/Downloads/PROJECT-COURSERA-CAPSTONE/DATA/Boroughs_london.csv',index_col=0)

In [3]:
boroughs

,Code,Borough,Inner/Outer,GLA_P2017,GLA_H2017,Latitude,Longitude
0,E09000001,City of London,Inner London,8800,5326,51.5155,-0.0922
1,E09000002,Barking and Dagenham,Outer London,209000,78188,51.5607,0.1557
2,E09000003,Barnet,Outer London,389600,151423,51.6252,-0.1517
3,E09000004,Bexley,Outer London,244300,97736,51.4549,0.1505
4,E09000005,Brent,Outer London,332100,121048,51.5588,-0.2817
5,E09000006,Bromley,Outer London,327900,140602,51.4039,0.0198
6,E09000007,Camden,Inner London,242500,107654,51.5290,-0.1255
7,E09000008,Croydon,Outer London,386500,159010,51.3714,-0.0977
8,E09000009,Ealing,Outer London,351600,132663,51.5130,-0.3089
9,E09000010,Enfield,Outer London,333000,130328,51.6538,-0.0799


From Wikipedia catch the List of neighbourhoods

In [4]:
file="https://en.wikipedia.org/wiki/List_of_areas_of_London"
neig=pd.read_html(file)[1]

In [5]:
neig.rename({'Location':'Neighbourhood',list(neig)[1]:'Borough'},axis='columns',inplace=True)
neig.columns

Index(['Neighbourhood', 'Borough', 'Post town', 'Postcode district',
       'Dial code', 'OS grid ref'],
      dtype='object')

In [6]:
neig_boroughs=neig[neig.columns[:2]]


In [7]:
def chain_digit(texto):
    try:
        ll= [k for k in texto if k.isdigit()]
    except:
        ll=[]
    return len(ll)

def remove_square_bracket(text):
    lc=chain_digit(text)
    if lc>0:
        return text.replace(text[-2-lc:],'  ')
    else:
        return text
    
def members_chain(text):
    text0=text.split(',')
    ll= len(text0)
    text1=[] 
    for ind in text0:
        text1.append(ind.strip())
    return ll,text1
        


In [8]:
neig_boroughs.head()

,Neighbourhood,Borough
0,Abbey Wood,"Bexley, Greenwich [7]"
1,Acton,"Ealing, Hammersmith and Fulham[8]"
2,Addington,Croydon[8]
3,Addiscombe,Croydon[8]
4,Albany Park,Bexley


Remove brackets of Borough List

In [9]:
kk=0
ss=len(neig_boroughs.index)-1
for ind in neig_boroughs.index:
    text=remove_square_bracket(neig_boroughs.loc[ind,'Borough'])
    ll,text0=members_chain(text)
    neig_boroughs=neig_boroughs.replace(neig_boroughs.loc[ind,'Borough'],text0[0])
    for kk in range(1,ll):
        ss=ss+1
        neig_boroughs.loc[ss]=[neig_boroughs.loc[ind,'Neighbourhood'],text0[kk]]


In [10]:
df=[['Southbank','Lambeth'],['Waterloo','Southwark'],
    ['Bermondsey','Southwark'],['Kings Cross','Camden']]
df=pd.DataFrame(df,columns=neig_boroughs.columns)
neig_boroughs=neig_boroughs.append(df,ignore_index=True)

In [11]:
neig_bor=neig_boroughs.sort_values(['Borough'],ascending=True)
neig_bor=neig_bor.replace('City','City of London')

In [12]:
neig_bor.shape


(563, 2)

In [13]:
filekml="/home/sedna/Downloads/PROJECT-COURSERA-CAPSTONE/DATA/areas-coordinates.kml"
from fastkml import kml
with open(filekml,'r') as f:
     doc=f.read()

from bs4 import BeautifulSoup   
soup=BeautifulSoup(doc,'html.parser')

In [14]:
print(soup.prettify())

<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://earth.google.com/kml/2.1">
 <document>
  <name>
   <![CDATA[Category:Areas of London]]>
  </name>
  <open>
   1
  </open>
  <placemark>
   <name>
    <![CDATA[Abbey Wood]]>
   </name>
   <point>
    <coordinates>
     0.1109,51.4864,0
    </coordinates>
   </point>
   <snippet>
   </snippet>
   <description>
    <![CDATA[<br>Source: Wikipedia article <a href="https://en.wikipedia.org/wiki/Abbey_Wood">Abbey Wood</a>]]>
   </description>
  </placemark>
  <placemark>
   <name>
    <![CDATA[Aberfeldy Village]]>
   </name>
   <point>
    <coordinates>
     -0.006,51.513,0
    </coordinates>
   </point>
   <snippet>
   </snippet>
   <description>
    <![CDATA[<br>Source: Wikipedia article <a href="https://en.wikipedia.org/wiki/Aberfeldy_Village">Aberfeldy Village</a>]]>
   </description>
  </placemark>
  <placemark>
   <name>
    <![CDATA[Acton, London]]>
   </name>
   <point>
    <coordinates>
     -0.270661,51.513519,0
    </coordin

In [15]:
List_placemark=[]
for link in soup.find_all(['placemark']):           
            List_placemark.append(link.get_text().split('\n'))
len(List_placemark)

707

In [16]:
List_description=[]
for link in soup.find_all(['description']):           
            List_description.append(link.get_text())
len(List_description)

707

In [17]:
List_diff=[link for tag in List_placemark for link in tag if link not in List_description]
List_diff

['',
 'Abbey Wood',
 '',
 '0.1109,51.4864,0',
 '',
 '',
 '',
 '',
 'Aberfeldy Village',
 '',
 '-0.006,51.513,0',
 '',
 '',
 '',
 '',
 'Acton, London',
 '',
 '-0.270661,51.513519,0',
 '',
 '',
 '',
 '',
 'Acton Green, London',
 '',
 '-0.262668,51.510515,0',
 '',
 '',
 '',
 '',
 'Acton Vale, London',
 '',
 '-0.258,51.511,0',
 '',
 '',
 '',
 '',
 'Addington, London',
 '',
 '-0.0305,51.3583,0',
 '',
 '',
 '',
 '',
 'Addiscombe',
 '',
 '-0.0663,51.381,0',
 '',
 '',
 '',
 '',
 'Adelphi, London',
 '',
 '-0.1225,51.5091666666667,0',
 '',
 '',
 '',
 '',
 'Agar Town',
 '',
 '-0.131944444444444,51.5388888888889,0',
 '',
 '',
 '',
 '',
 'Albany Park, Bexley',
 '',
 '0.1026,51.4264,0',
 '',
 '',
 '',
 '',
 'Albertopolis',
 '',
 '-0.176666666666667,51.4980555555556,0',
 '',
 '',
 '',
 '',
 'Aldborough Hatch',
 '',
 '0.10102165,51.58355,0',
 '',
 '',
 '',
 '',
 'Aldwych',
 '',
 '-0.1167,51.5132,0',
 '',
 '',
 '',
 '',
 'Alperton',
 '',
 '-0.2994,51.5405,0',
 '',
 '',
 '',
 '',
 'Anerley',
 '',
 '-0.0

In [18]:
List_Area_name=[]
List_latitude=[]
List_longitude=[]
for tag0 in List_diff:
    if tag0 !='':
        if chain_digit(tag0)>0:
            tag=tag0.split(',')        
            List_latitude.append(float(tag[1]))
            List_longitude.append(float(tag[0]))
        else:
            List_Area_name.append(tag0)
            
#nn=(len(List_Area_name)-1)
for nn in range(len(List_Area_name)):
    print("%s%15.5f%15.5f" %(List_Area_name[nn].center(30),List_latitude[nn],List_longitude[nn]))
      

          Abbey Wood                 51.48640        0.11090
      Aberfeldy Village              51.51300       -0.00600
        Acton, London                51.51352       -0.27066
     Acton Green, London             51.51051       -0.26267
      Acton Vale, London             51.51100       -0.25800
      Addington, London              51.35830       -0.03050
          Addiscombe                 51.38100       -0.06630
       Adelphi, London               51.50917       -0.12250
          Agar Town                  51.53889       -0.13194
     Albany Park, Bexley             51.42640        0.10260
         Albertopolis                51.49806       -0.17667
       Aldborough Hatch              51.58355        0.10102
           Aldwych                   51.51320       -0.11670
           Alperton                  51.54050       -0.29940
           Anerley                   51.41470       -0.06700
        Angel, London                51.53210       -0.10660
          Aperfield     

In [19]:
for ind in range(len(List_Area_name)):
    ind0=List_Area_name[ind].split(',')
    for kk in ind0:
         if kk.strip()=='London':
                List_Area_name[ind]=ind0[0]

In [20]:

geolocator=Nominatim(user_agent="RUPERT")

nn=0
for ii in neig_bor.index:
    if neig_bor.loc[ii,'Neighbourhood'] not in  List_Area_name:
        nn+=1
        place=neig_bor.loc[ii,'Neighbourhood'] .split('(')[0]
        address=place +','+ neig_bor.loc[ii,'Borough'] + ','+ 'London, UK'
        try:
            g=geolocator.geocode(address)
            print(ii,neig_bor.loc[ii,'Neighbourhood'],g.latitude,g.longitude)
            List_Area_name.append(neig_bor.loc[ii,'Neighbourhood'])
            List_latitude.append(g.latitude)
            List_longitude.append(g.longitude)
        except:
            print(ii,neig_bor.loc[ii,'Neighbourhood'],"is not in official Neighbourhood List") 

300 Marks Gate 51.5863692 0.1348482
468 Totteridge 51.6331335 -0.2008636
101 Church End 51.601117 -0.1918901
94 Childs Hill 51.5629822 -0.1972494
322 New Barnet 51.6487843 -0.1729127
73 Burroughs, The 51.5863721 -0.2303106
181 Friern Barnet 51.6128484 -0.158614
25 Barnet (also Chipping Barnet, High Barnet) 51.6487843 -0.1729127
24 Barnet Gate 51.6418265 -0.2429855
334 North End is not in official Neighbourhood List
109 Colyers 51.4710947 0.1644709
4 Albany Park 51.4361034 0.1259205
507 West Heath 51.4808745 0.1273574
479 Upper Ruxley 51.4102386 0.1436717
44 Bexley (also Old Bexley, Bexley Village) 51.4416793 0.150488
45 Bexleyheath (also Bexley New Town) 51.4634854 0.1480552
481 Upton 51.4524572 0.1275711
517 Widmore (also Widmore Green) 51.4023963 0.0351602
424 Southborough 51.3899151 0.0446175
129 Cudham 51.3163662 0.0748332
231 Hayes 51.3777836 0.0103002
233 Hazelwood 51.3333728 0.0749494
369 Plaistow 51.4151484 0.0171595
554 Sydenham (also Lower Sydenham, Upper Sydenham) 51.4028046

In [21]:
Locations=pd.DataFrame(columns=['Neighbourhood','Latitude','Longitude'])

In [22]:
Locations['Neighbourhood']=List_Area_name

In [23]:
Locations['Latitude']=List_latitude
Locations['Longitude']=List_longitude

In [24]:
Locations.head()

,Neighbourhood,Latitude,Longitude
0,Abbey Wood,51.486400,0.110900
1,Aberfeldy Village,51.513000,-0.006000
2,Acton,51.513519,-0.270661
3,Acton Green,51.510515,-0.262668
4,Acton Vale,51.511000,-0.258000


In [25]:
Locations.shape 

(774, 3)

In [26]:
neighbour_hoods=pd.merge(neig_bor,Locations,on='Neighbourhood')



Filter boroughs list in neighbour_hoods against oficial boroughs list

In [27]:
def drop_bor(nn,bb,colA):
     kk=(nn.columns==colA).tolist().index(True)
     pp=(bb.columns==colA).tolist().index(True)
     
     for i in nn.index:
            if not any(x==nn[nn.columns[kk]][i].lstrip() for x in bb[bb.columns[pp]].tolist()):
                print(i,nn[nn.columns[kk]][i])
                nn=nn.drop([i])
     return nn

In [28]:
neighbourhoods=drop_bor(neighbour_hoods,boroughs,'Borough')
neighbourhoods.shape

169 Camden and Islington
198 Dartford
292 Haringey and Barnet
372 Islington & City
381 Kensington and ChelseaHammersmith and Fulham


(563, 4)

In [29]:
cols=list(neighbourhoods)
cols[0],cols[1]=cols[1],cols[0]
neighbourhoods.columns=cols
neighbourhoods_copy=neighbourhoods.copy()
neighbourhoods[cols[0]]=neighbourhoods[cols[1]]
neighbourhoods[cols[1]]=neighbourhoods_copy[cols[0]]

In [30]:
neighbourhoods.reset_index(drop=True)

,Borough,Neighbourhood,Latitude,Longitude
0,Barking and Dagenham,Rush Green,51.565500,0.176000
1,Barking and Dagenham,Marks Gate,51.586369,0.134848
2,Barking and Dagenham,Becontree Heath,51.563595,0.152614
3,Barking and Dagenham,Becontree,51.548700,0.142700
4,Barking and Dagenham,Barking,51.540000,0.080000
...,...,...,...,...
558,Westminster,Millbank,51.491500,-0.129500
559,Westminster,Maida Vale,51.527400,-0.189900
560,Westminster,St James's,51.508500,-0.133000
561,Westminster,Covent Garden,51.512874,-0.122544


In [31]:
neighbourhoods.shape

(563, 4)

In [32]:
neighbourhoods.to_csv('/home/sedna/Downloads/PROJECT-COURSERA-CAPSTONE/DATA/neighbourhoods_london.csv')

Examine the dataframe resulting

Count the number of neighbourhoods corresponding to each borough


In [33]:
number_neig=neighbourhoods.groupby('Borough').size()
number_neig=pd.DataFrame(number_neig, index=boroughs.Borough,columns=["Neighbourhoods"])
print(number_neig)

                        Neighbourhoods
Borough                               
City of London                       4
Barking and Dagenham                 9
Barnet                              36
Bexley                              30
Brent                               22
Bromley                             39
Camden                              20
Croydon                             22
Ealing                              13
Enfield                             20
Greenwich                           17
Hackney                             18
Hammersmith and Fulham              11
Haringey                            16
Harrow                              15
Havering                            21
Hillingdon                          17
Hounslow                            15
Islington                           16
Kensington and Chelsea               8
Kingston upon Thames                14
Lambeth                             13
Lewisham                            21
Merton                   

Show a table with the information of number boroughs and its number of neighbourhoods

In [34]:
neighbourhoods.Borough.count()

563

In [35]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(boroughs.shape[0],
       number_neig.Neighbourhoods.sum()
    )
)


The dataframe has 33 boroughs and 563 neighborhoods.


Use geopy library to get the latitude and longitude values of London. Define an instance of the geocoder, with the user_agent named "to_explorer"

In [36]:
address = 'London,UK'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of  London City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of  London City are 51.5073219, -0.1276474.


Create a map of London with neighbourhoods superimposed on top

In [51]:

# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the neighbourhoods
import random
def get_colors(n):
    return list(map(lambda i: "#" + "%06x" % random.randint(0, 0xFFFFFF),range(n)))
colors=get_colors(boroughs.shape[0])

# add markers to map
for lat, lng, borough,neighbourhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], 
    neighbourhoods['Borough'],neighbourhoods['Neighbourhood']):
            label = '{}, {}'.format(neighbourhood, borough)
            label = folium.Popup(label, parse_html=True)
            folium.CircleMarker(
                [lat, lng],
                radius=5,
                popup=label,
                color=colors[boroughs[boroughs.Borough==borough].index[0]],
                fill=True,
                fill_color='white',
                fill_opacity=0.9,
                parse_html=False).add_to(map_london)  
map_london 


In [41]:
map_london.save("/home/sedna/Downloads/PROJECT-COURSERA-CAPSTONE/RESULTS/map_london_neighbourhoods_by_borough.html")

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

Define Foursquare Credentials and Version

In [42]:
CLIENT_ID = 'X3INNHZFS0RE4YAMNP5CCG4UVUP4S5K1S4NZIIXKHN2UNXJW' # your Foursquare ID
CLIENT_SECRET = '20EOF0IP0MALLD0RO0MM1QOCLEHXC35L4U3JHMETTEF24YJR' # your Foursquare Secret

VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X3INNHZFS0RE4YAMNP5CCG4UVUP4S5K1S4NZIIXKHN2UNXJW
CLIENT_SECRET:20EOF0IP0MALLD0RO0MM1QOCLEHXC35L4U3JHMETTEF24YJR


Explore Neighbourhoods in London.
Let's create a function that extract all information  from all the neighborhoods in London

In [43]:
def getNearbyVenues(names_b,names, latitudes, longitudes, radius=500,LIMIT=100):
    
    venues_list=[]
    for name_b,name, lat, lng in zip(names_b,names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()
        try:
            venues=results["response"]['groups'][0]['items']
            # return only relevant information for each nearby venue
            venues_list.append([(
                name_b,
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name'],
                v['venue']['categories'][0]['icon']['prefix'].split('/')[5]) for v in venues])
        except:
               print("Venue no has been found")            
                
        
    #venues[0]['venue']['categories'][0]['icon']['prefix'].split('/')[5]
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough','Neighbourhood', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue category',
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called London_neig_venues.

In [44]:

London_neig_venues = getNearbyVenues(names_b=neighbourhoods['Borough'],names=neighbourhoods['Neighbourhood'],
                                   latitudes=neighbourhoods['Latitude'],
                                   longitudes=neighbourhoods['Longitude']
                                  )

print(London_neig_venues.shape)
London_neig_venues.head()


Rush Green
Marks Gate
Becontree Heath
Becontree
Barking
Chadwell Heath
Chadwell Heath
Dagenham
Creekmouth
Castle Green
Brent Cross
Colney Hatch
East Barnet
Edgware
Colindale
Cockfosters
Cockfosters
Totteridge
Church End
Church End
North Finchley
Temple Fortune
Childs Hill
East Finchley
Mill Hill
Monken Hadley
New Barnet
The Hyde
Burroughs, The
Friern Barnet
Barnet (also Chipping Barnet, High Barnet)
Golders Green
Finchley
Grahame Park
Burnt Oak
Woodside Park
Arkley
Osidge
The Hale
Oakleigh Park
Brunswick Park
West Hendon
Hampstead Garden Suburb
Cricklewood
Cricklewood
Cricklewood
New Southgate
Barnet Gate
Whetstone
Hendon
Northumberland Heath
North Cray
Abbey Wood
Abbey Wood
Lessness Heath
Lamorbey
Crayford
Crook Log
Crossness
Foots Cray
Falconwood
Falconwood
Erith
Longlands
Colyers
East Wickham
Sidcup
Albany Park
Barnehurst
West Heath
Barnes Cray
Welling
Ruxley
Ruxley
Upper Ruxley
Thamesmead
Belvedere
Bexley (also Old Bexley, Bexley Village)
Bexleyheath (also Bexley New Town)
Blackfen

,Borough,Neighbourhood,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue category,Venue Category
0,Barking and Dagenham,Rush Green,51.5655,0.176,Co-op Food,51.565820,0.175966,Grocery Store,shops
1,Barking and Dagenham,Rush Green,51.5655,0.176,BP,51.565801,0.176862,Gas Station,shops
2,Barking and Dagenham,Rush Green,51.5655,0.176,Costa Coffee,51.568029,0.180212,Coffee Shop,food
3,Barking and Dagenham,Rush Green,51.5655,0.176,Lynne Davis Wedding and Events,51.565908,0.178294,Flower Shop,shops
4,Barking and Dagenham,Rush Green,51.5655,0.176,Go Shop,51.568454,0.180030,Convenience Store,shops


In [45]:
London_neig_venues.to_csv('/home/sedna/Downloads/PROJECT-COURSERA-CAPSTONE/DATA/london_borough_neig_venues.csv')

In [49]:
London_neig_venues=London_neig_venues.drop(['Borough'],axis=1)


,Neighbourhood,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue category,Venue Category
0,Rush Green,51.5655,0.176,Co-op Food,51.565820,0.175966,Grocery Store,shops
1,Rush Green,51.5655,0.176,BP,51.565801,0.176862,Gas Station,shops
2,Rush Green,51.5655,0.176,Costa Coffee,51.568029,0.180212,Coffee Shop,food
3,Rush Green,51.5655,0.176,Lynne Davis Wedding and Events,51.565908,0.178294,Flower Shop,shops
4,Rush Green,51.5655,0.176,Go Shop,51.568454,0.180030,Convenience Store,shops
...,...,...,...,...,...,...,...,...
12457,Paddington,51.5172,-0.173,Hotel Edward,51.514241,-0.175050,Hotel,travel
12458,Paddington,51.5172,-0.173,Mitre House Hotel,51.514350,-0.174338,Hotel,travel
12459,Paddington,51.5172,-0.173,MARA House,51.514404,-0.175102,Hotel,travel
12460,Paddington,51.5172,-0.173,GoNative Sussex Gardens,51.513982,-0.175877,Hotel,travel


In [50]:
London_neig_venues.to_csv('/home/sedna/Downloads/PROJECT-COURSERA-CAPSTONE/DATA/london_neig_venues.csv')